In [5]:
import os 
import sys
import json 
import pandas as pd 
import re
BASE_PATH = "/mnt/e/Personal/Samarth/repository/DMAC_ChatGPT/"

sys.path.append(BASE_PATH+'/src/conversation_bot')
from data_orchestrator.data_processing import DataOrchestrator


/home/samartht/anaconda3/envs/pylangchain/lib/python3.10/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


In [6]:
data_path = os.path.join(BASE_PATH,"data/Audio/")

audio_files = [os.path.join(data_path,fname) for fname in os.listdir(data_path)]
print(audio_files)

['/mnt/e/Personal/Samarth/repository/DMAC_ChatGPT/data/Audio/01fe7ec7-5fec-46cc-b3be-7946f5a84085.mp3', '/mnt/e/Personal/Samarth/repository/DMAC_ChatGPT/data/Audio/036d4b30-b163-41ff-a1f7-1d5a96a71f55.mp3', '/mnt/e/Personal/Samarth/repository/DMAC_ChatGPT/data/Audio/03dac286-7d7e-414c-93d2-6da513d5546f.mp3', '/mnt/e/Personal/Samarth/repository/DMAC_ChatGPT/data/Audio/03ed77b4-a442-4bd0-b40e-493a59ced856.mp3', '/mnt/e/Personal/Samarth/repository/DMAC_ChatGPT/data/Audio/0468776d-3e38-446a-9303-ea1d6e03bbbd.mp3', '/mnt/e/Personal/Samarth/repository/DMAC_ChatGPT/data/Audio/04c89f3c-4149-4a46-a26e-ed56cf65c74d.mp3', '/mnt/e/Personal/Samarth/repository/DMAC_ChatGPT/data/Audio/055f5999-61b6-4f86-8727-5920d0e4fde6.mp3', '/mnt/e/Personal/Samarth/repository/DMAC_ChatGPT/data/Audio/05d4154d-a644-4329-9ca5-0b2aaade868d.mp3', '/mnt/e/Personal/Samarth/repository/DMAC_ChatGPT/data/Audio/05f3bb3d-52f3-4c39-a0ef-8e5d7fc2ff52.mp3', '/mnt/e/Personal/Samarth/repository/DMAC_ChatGPT/data/Audio/062adf52-8c9

In [8]:
data_orchestrator = DataOrchestrator(cnf_path=os.path.join(BASE_PATH,"conf/conf.cnf"))
data_orchestrator.process_records(audio_files[:1])

small whisper model loaded 


100%|██████████| 1/1 [00:47<00:00, 47.14s/it]


0